In [6]:
%pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.1/230.1 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 40.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 8.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [1]:
# Read libraries

# Data wrangling
import pandas as pd
import numpy as np
import re
from shapely import wkt
import fiona

# Visualization 
import missingno as msno
import matplotlib
import mapclassify
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import seaborn as sns
import geopandas as gpd


# Geostatistics
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW
import libpysal as ps
import matplotlib.pyplot as plt
import seaborn as sns
from pysal.lib import weights
from pysal.model import spreg
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.compat import lzip
from esda.moran import Moran_Local
from splot.esda import lisa_cluster
from pysal.lib import weights
from splot.libpysal import plot_spatial_weights
from spreg import OLS
from spreg import MoranRes
from spreg import ML_Lag
from spreg import ML_Error

# Machine learning 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Look at this libraries later
#from keras.models import Sequential
#from keras.layers import Dense, Dropout
#from keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings('ignore')

/Users/albertcortbanke/miniconda3/envs/msngdata/lib/python3.9/site-packages/libpysal/cg/alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
/Users/albertcortbanke/miniconda3/envs/msngdata/lib/python3.9/site-packages/libpysal/cg/alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
/Users/albe

In [3]:
# Read the GeoParquet file without one-hot sampling
gdf_no_oh = gpd.read_parquet('gdf_no_ohe.parquet')

In [4]:
gdf_no_oh

,year,countries,region,pf_ss_disappearances_disap,pf_ss_disappearances_violent,pf_ss_disappearances_violent_data,pf_ss_disappearances_organized,pf_ss_disappearances_fatalities,pf_ss_disappearances_fatalities_data,pf_ss_disappearances_injuries,...,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,country_code,geometry,globalNorth,ef_score,hf_score,pf_expression_bti,pf_rol_civil
0,2020.0,Albania,Eastern Europe,10.0,10.000000,0.0,7.500000,10.000000,0.0,10.000000,...,7.175250,6.698118,7.112958,ALB,"POLYGON ((20.07142 42.56091, 20.10208 42.53347...",1.0,0,0,0,0
1,2020.0,Algeria,Middle East & North Africa,10.0,9.687083,25.0,5.000000,10.000000,0.0,10.000000,...,7.029528,5.686753,5.778953,DZA,"POLYGON ((8.62203 36.94137, 8.63222 36.88194, ...",0.0,0,0,0,0
2,2020.0,Angola,Sub-Saharan Africa,10.0,9.582498,25.0,7.500000,9.736578,5.0,9.971733,...,6.782923,5.696942,6.227545,AGO,"MULTIPOLYGON (((23.98621 -10.87046, 23.98805 -...",0.0,0,0,0,0
3,2020.0,Argentina,Latin America & the Caribbean,5.0,10.000000,0.0,7.500000,9.925379,2.0,10.000000,...,6.508295,6.225232,5.490538,ARG,"MULTIPOLYGON (((-68.64312 -54.88861, -68.63723...",0.0,0,0,0,0
4,2020.0,Armenia,Caucasus & Central Asia,10.0,10.000000,0.0,7.500000,10.000000,0.0,10.000000,...,7.040738,7.124727,7.756333,ARM,"POLYGON ((46.54038 38.87559, 46.51639 38.87804...",1.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3481,2000.0,Venezuela,Latin America & the Caribbean,10.0,10.000000,0.0,8.231718,10.000000,0.0,10.000000,...,0.315142,4.692328,5.531397,VEN,"MULTIPOLYGON (((-61.18507 8.49674, -61.19417 8...",0.0,0,0,0,1
3482,2000.0,Vietnam,South Asia,10.0,10.000000,0.0,7.964260,10.000000,0.0,10.000000,...,0.000000,3.861879,4.416768,VNM,"MULTIPOLYGON (((104.08288 10.36486, 104.08663 ...",0.0,0,0,0,1
3483,2000.0,Yemen,Middle East & North Africa,5.0,10.000000,0.0,7.044853,8.170079,19.0,7.932996,...,6.988352,5.658095,6.684552,YEM,"MULTIPOLYGON (((52.23416 12.20111, 52.27888 12...",0.0,1,1,0,1
3484,2000.0,Zambia,Sub-Saharan Africa,10.0,10.000000,0.0,7.552838,10.000000,0.0,9.827262,...,8.520369,8.020982,6.799421,ZMB,"POLYGON ((32.94040 -9.40508, 32.93944 -9.41583...",0.0,0,0,0,1


## Feature scaling

Feature scaling is a method used to standardize the range of independent variables or features of data. In data processing, it is also known as data normalization and is generally performed during the data preprocessing step.

Note: Before doing feature scaling, make sure to split your data into train and test sets, to avoid data leakage. Data leakage can happen when information from outside your training dataset is used to create the model. This can cause the model to overestimate its accuracy.

First, let's do a **Train-Test Split**:

In [5]:
# Creating the X and y variables
X = gdf_no_oh.drop(['ef_score', 'hf_score', 'pf_expression_bti', 'pf_rol_civil'], axis=1)  # Features
y = gdf_no_oh[['ef_score', 'hf_score', 'pf_expression_bti', 'pf_rol_civil']]  # Targets

# Perform a chronological train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [7]:
# Select only numeric features
X_train_numeric = X_train.select_dtypes(include=np.number)
X_test_numeric = X_test.select_dtypes(include=np.number)

# Initialize a new scaling object for normalizing input variables
scaler = StandardScaler()

# Fit the scaler object to the training data
scaler.fit(X_train_numeric)

# Apply the scaler object to the training and test sets
X_train_scaled = scaler.transform(X_train_numeric)
X_test_scaled = scaler.transform(X_test_numeric)

# Replace the original numeric columns with the scaled ones
X_train[X_train_numeric.columns] = X_train_scaled
X_test[X_test_numeric.columns] = X_test_scaled

In [8]:
X_train

,year,countries,region,pf_ss_disappearances_disap,pf_ss_disappearances_violent,pf_ss_disappearances_violent_data,pf_ss_disappearances_organized,pf_ss_disappearances_fatalities,pf_ss_disappearances_fatalities_data,pf_ss_disappearances_injuries,...,ef_regulation_business_burden,ef_regulation_business_start,ef_regulation_business_impartial,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,country_code,geometry,globalNorth
0,1.615007,Albania,Eastern Europe,0.492159,0.330563,-0.100248,0.224420,0.411347,-0.162733,0.400277,...,0.285562,0.749013,0.084699,-1.141352,0.153431,0.236609,0.138248,ALB,"POLYGON ((20.07142 42.56091, 20.10208 42.53347...",1.400416
1,1.615007,Algeria,Middle East & North Africa,0.492159,0.195341,-0.090143,-0.683959,0.411347,-0.162733,0.400277,...,-1.288962,0.383843,-1.354790,0.628961,0.082556,-0.560355,-1.140556,DZA,"POLYGON ((8.62203 36.94137, 8.63222 36.88194, ...",-0.714074
2,1.615007,Angola,Sub-Saharan Africa,0.492159,0.150146,-0.090143,0.224420,0.315619,-0.154138,0.389133,...,-0.918486,-0.095452,-0.460370,0.148492,-0.037385,-0.552327,-0.710526,AGO,"MULTIPOLYGON (((23.98621 -10.87046, 23.98805 -...",-0.714074
3,1.615007,Argentina,Latin America & the Caribbean,-1.166200,0.330563,-0.100248,0.224420,0.384230,-0.159295,0.400277,...,0.100324,0.612795,0.619246,-1.082562,-0.170956,-0.136029,-1.417036,ARG,"MULTIPOLYGON (((-68.64312 -54.88861, -68.63723...",-0.714074
4,1.615007,Armenia,Caucasus & Central Asia,0.492159,0.330563,-0.100248,0.224420,0.411347,-0.162733,0.400277,...,-0.084915,0.850059,0.195186,0.927725,0.088008,0.572781,0.755000,ARM,"POLYGON ((46.54038 38.87559, 46.51639 38.87804...",1.400416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,-1.682581,Niger,Sub-Saharan Africa,0.492159,0.330563,-0.100248,0.246972,0.411347,-0.162733,0.400277,...,0.431254,-3.196469,0.002623,-3.404105,0.055297,-1.221983,-1.660098,NER,"POLYGON ((14.99789 23.00059, 15.00694 22.95944...",-0.714074
2436,-1.682581,Nigeria,Sub-Saharan Africa,0.492159,0.330563,-0.100248,0.002343,-0.674440,0.273884,0.348025,...,0.544123,-0.440960,-1.186428,-2.466463,-3.336398,-1.750003,0.446448,NGA,"MULTIPOLYGON (((7.20361 4.38056, 7.18028 4.377...",-0.714074
2437,-1.682581,North Macedonia,Eastern Europe,0.492159,0.330563,-0.100248,0.337280,0.411347,-0.162733,0.400277,...,-0.106116,0.505229,0.466669,-2.085472,1.003925,-0.106614,0.622053,MKD,"POLYGON ((22.36528 42.32388, 22.40778 42.27944...",1.400416
2438,-1.682581,Norway,Western Europe,0.492159,0.330563,-0.100248,1.075797,0.411347,-0.162733,0.400277,...,-1.036365,0.692694,1.590480,-0.521123,1.052992,0.505962,0.219201,NOR,"MULTIPOLYGON (((5.28778 59.21889, 5.25597 59.1...",1.400416
